In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix , roc_auc_score, roc_curve, precision_recall_curve, auc
import pickle
import numpy as np
import pandas as pd
import matplotlib as mpl
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import os
from keras.utils import to_categorical
import pathlib
import glob
tf.test.is_gpu_available()
os.chdir('/data/swamyvs/pacbio_testing/')

In [2]:
class SimpleDataObj:
    def __init__(self, X_df, labs,kmer_size, one_label, zero_label, y_format='tf'):
        positive_cases=['all', 'stringtie-pacbio', 'scallop-pacbio']
        X_df_labeled=pd.merge(left=labs, right=X_df, left_on='transcript_id', right_on='transcript_id')
        X_data=np.asarray(X_df_labeled.iloc[:,3:])#drop the first 3 columns
        Y_vec=np.asarray(X_df_labeled['target_label'])
        self.Y_origin=X_df_labeled.iloc[:,:3]
        self.vec_y=Y_vec
        
        X_train, self.X_val, Y_train_labs, Y_val_labs= train_test_split(X_data,labs,test_size=.2, random_state=42, stratify=Y_vec)
        self.X_train, self.X_test, Y_train_labs, Y_test_labs=train_test_split(X_train,Y_train_labs,test_size=.2, 
                                                                                  random_state=42,stratify=Y_train_labs['target_label'])

        if y_format == 'tf':
            self.Y_val=to_categorical(Y_val_labs['target_label'])
            self.Y_train=to_categorical(Y_train_labs['target_label'])
            self.Y_test=to_categorical(Y_test_labs['target_label'])
        else:
            self.Y_val=np.asarray(Y_val_labs['target_label'])
            self.Y_train=np.asarray(Y_train_labs['target_label'])
            self.Y_test=np.asarray(Y_test_labs['target_label'])
            
        self.Y_val_labs=Y_val_labs[['transcript_id','intersection_case']] 
        self.Y_train_labs=Y_train_labs[['transcript_id','intersection_case']]
        self.Y_test_labs=Y_test_labs[['transcript_id','intersection_case']]
            
        self.y_format=y_format
        self.one_label=one_label
        self.zero_label=zero_label
    def summary(self):
        tr_len=len(self.X_train)
        ts_len=len(self.X_test)
        v_len= len(self.X_val)
        print(f'Training size: {tr_len}\nvalidation size: {v_len}\ntesting size: {ts_len}')
        print(f'{self.one_label} count: {np.count_nonzero(self.vec_y == 1)}\n{self.zero_label} count : {np.count_nonzero(self.vec_y == 0)}')

        
class RnnDataObj:
    def __init__(self,  X_df, labs,kmer_size,block_size, one_label, zero_label):
        positive_cases=['all', 'stringtie-pacbio', 'scallop-pacbio']
        X_df_labeled=pd.merge(left=labs, right=X_df, left_on='transcript_id', right_on='transcript_id')
        X_data=np.asarray(X_df_labeled.iloc[:,3:])#drop the first 3 columns
        
        assert X_data.shape[1] % block_size == 0
        block_dim=int(X_data.shape[1] / block_size)
        X_data=np.asarray( [vec.reshape(block_size, block_dim) for vec in X_data])
        Y_vec=np.asarray(X_df_labeled['target_label'])
        
        self.Y_origin=X_df_labeled.iloc[:,:3]
        self.vec_y=Y_vec

        X_train, self.X_val, Y_train_labs, Y_val_labs= train_test_split(X_data,labs,test_size=.2, random_state=42, stratify=Y_vec)
        self.Y_val=to_categorical(Y_val_labs['target_label'])
        self.Y_val_labs=Y_val_labs[['transcript_id','intersection_case']]
        self.X_train, self.X_test, Y_train_labs, Y_test_labs=train_test_split(X_train,Y_train_labs,test_size=.2, 
                                                                              random_state=42,stratify=Y_train_labs['target_label'])
        self.Y_train=to_categorical(Y_train_labs['target_label'])
        self.Y_train_labs=Y_train_labs[['transcript_id','intersection_case']]
        self.Y_test=to_categorical(Y_test_labs['target_label'])
        self.Y_test_labs=Y_test_labs[['transcript_id','intersection_case']]

        self.one_label=one_label
        self.zero_label=zero_label
    def summary(self):
        tr_len=len(self.X_train)
        ts_len=len(self.X_test)
        v_len= len(self.X_val)
        print(f'Training size: {tr_len}\nvalidation size: {v_len}\ntesting size: {ts_len}')
        print(f'{self.one_label} count: {np.count_nonzero(self.vec_y == 1)}\n{self.zero_label} count : {np.count_nonzero(self.vec_y == 0)}')
        

def model_results(Y_true, Y_pred_class, Y_pred_prob,kmer_size, edim, model_name, outdir):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    data_name=[f'kmer={str(kmer_size)}_edim={str(edim)}_model={model_name}']
    fpr, tpr, thresholds=roc_curve(Y_true, Y_pred_prob)
    AUC=roc_auc_score(Y_true, Y_pred_class)
    # Plot ROC curve
    plt.subplot(2,1,1)
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % AUC)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate or (1 - Specifity)')
    plt.ylabel('True Positive Rate or (Sensitivity)')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")


    pre, rec, thresholds = precision_recall_curve(Y_true, Y_pred_prob)
    AUC = auc(rec, pre)
    plt.subplot(2,1,2)
    plt.plot(rec, pre, label=' Prec/Rec (area = %0.2f)' % ( AUC))
    plt.plot([1, 1], [1, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall plot')
    plt.legend(loc="lower right")


    plt.suptitle(f'{data_name[0]} ROC and PRC')
    #plt.show()
    plt.savefig(outdir + data_name[0] + '_roc_prc.png' )
    plt.close()
    labs=['not_transcript', 'transcript'] 
    cr_dict=classification_report(y_pred=Y_pred_class, y_true=Y_true, output_dict=True)
    data_name=[f'kmer={str(kmer_size)}_edim={str(edim)}_model={model_name}']
    tm=['precision', 'recall','f1-score']
    zero_line=[str(cr_dict['0'][key])  for key in tm]
    one_line = [str(cr_dict['1'][key])  for key in tm]
    accuracy=str(cr_dict['accuracy'])
    out_cs_line= ','.join(data_name + zero_line + one_line +[accuracy]) + '\n'
    return(out_cs_line)

def train_sk_model(dat, model):
    model.fit(dat.X_train, dat.Y_train)
    Y_pred_class=model.predict(dat.X_test)
    Y_pred_prob=model.predict_proba(dat.X_test)[:,1]
    return model, Y_pred_class, Y_pred_prob
def train_tf_model(obj, model, batch_size, nepochs, model_name, outdir):
    history=model.fit(obj.X_train, obj.Y_train, epochs=nepochs, batch_size=batch_size, 
            validation_data=(obj.X_val, obj.Y_val), verbose=0)
    metrics =  ['loss', 'auc', 'accuracy']
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2,3,n+1)
        plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
                 color=colors[0], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
            plt.ylim([0.7,1])
        else:
            plt.ylim([0,1])

        plt.legend()
    plt.suptitle(f'{model_name} ROC and PRC')
    plt.savefig(outdir + f'{model_name}_training_metrics.png')
    plt.close()
    pred=[p[1] for p in  model.predict(obj.X_test)]
    pred_class = np.int64([p > .5 for p in pred])
    true_class=[int(i[1]) for i in obj.Y_test]
    return model, pred_class, pred
METRICS = [keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),]
def denseModel(edim):
    model=keras.Sequential()
    model.add(keras.layers.Dense(edim,activation='relu', kernel_initializer='he_uniform'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(.5))
        
    model.add(keras.layers.Dense(int(edim/4),  activation='relu', kernel_initializer='he_uniform'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(.5))
    
    if edim > 300:
        model.add(keras.layers.Dense(50,  activation='relu', kernel_initializer='he_uniform'))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dropout(.5))
    
    model.add(keras.layers.Dense(10,activation='relu', kernel_initializer='he_uniform'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(.5))
    
    model.add(keras.layers.Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=METRICS)
    return(model)
def lstmDenseModel(input_dim):
    NUM_UNITS=150
    model=keras.Sequential()
    model.add(keras.layers.Input(shape=input_dim))
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=NUM_UNITS)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(150,activation='relu', kernel_initializer='he_uniform'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(.5))
    model.add(keras.layers.Dense(50,  activation='relu', kernel_initializer='he_uniform'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(.5))
    model.add(keras.layers.Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=METRICS)

    return(model)



In [3]:
def run_data_through_models(X_file_name, handle):
    '''
    load data in sklearn, tf, and rnn format
    run each model through the data and 
        output f1,prec,rec, for each class, and accuracy
    make plots and save
    save model
    '''
    fn=X_file_name.split('/')[-1]
    kmer_size=int(fn.split('_')[4])
    edim=int(fn.split('_')[5].split('-')[1].split('.')[0])
    one_label='transcript'
    zero_label='not_transcript'
    lab_file='testing/all_RPE_loose_target_tx.tsv'
    #lab_file='data/gtf_info/all_RPE_loose_target_tx.tsv'
    positive_cases=['all', 'stringtie-pacbio', 'scallop-pacbio']
    mpl.rcParams['figure.figsize'] = (12, 10)
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    
    out_dir=f'data/model_experiments/kmer-{str(kmer_size)}_edim-{str(edim)}/'
    pathlib.Path(out_dir).mkdir(exist_ok=True)
    X_df=pd.read_csv(X_file_name,names=['transcript_id']+ list(range(edim)))
    labs=(pd
        .read_csv(lab_file, sep= '\t', names=['transcript_id', 'intersection_case'] )
        .assign(target_label=lambda x: np.where(x['intersection_case'].isin(positive_cases),1,0 )) )
    skl_obj=SimpleDataObj(X_df, labs, kmer_size, one_label, zero_label, 'sk')
    rf_model=RandomForestClassifier(n_estimators=100, random_state=32, n_jobs=32)
    rf_model_trained, rf_pred_class, rf_pred_prob=train_sk_model(skl_obj, rf_model)
    rf_outline=model_results(skl_obj.Y_test, rf_pred_class, rf_pred_prob, kmer_size, edim,'random_forest', out_dir)
    handle.write(rf_outline)
    
    wide_block_size = int(edim / 10)
    long_block_size = 10
    all_tf_data=[SimpleDataObj(X_df, labs, kmer_size, one_label, zero_label, 'tf'),
                 RnnDataObj(X_df, labs,kmer_size,wide_block_size, one_label, zero_label),
                 RnnDataObj(X_df, labs,kmer_size,long_block_size, one_label, zero_label)
                ]
    all_tf_models=[denseModel(edim),
                lstmDenseModel(all_tf_data[1].X_train.shape[1:]),
                lstmDenseModel(all_tf_data[2].X_train.shape[1:])
               ]
    tf_model_names=['Dense', 'LSTM-wide', 'LSTM-long']
    dense_batch_size=int(all_tf_data[0].X_train.shape[0] / 8)
    rnn_batch_size = int(all_tf_data[1].X_train.shape[0] / 128)
    batch_sizes=[dense_batch_size,rnn_batch_size, rnn_batch_size ]
    
    for i in range(len(all_tf_data)) :
        c_model=all_tf_models[i]
        c_data=all_tf_data[i]
        c_bs=batch_sizes[i]
        c_model_name=tf_model_names[i]
        trained_model, Y_pred_class, Y_pred_prob = train_tf_model(c_data, c_model, c_bs, 250, c_model_name, out_dir)
        Y_true=[int(i[1]) for i in c_data.Y_test]
        out_line=model_results(Y_true, Y_pred_class, Y_pred_prob, kmer_size, edim,c_model_name, out_dir)
        handle.write(out_line)
        c_model.save( f'{out_dir}{c_model_name}_trained.h5')
        
        

In [4]:
#with open('testing/model_res.out', 'w+') as tout:
#    run_data_through_models('testing/all_RPE_loose_kmers_8_dims-100.csv.gz', tout)

In [9]:
with open('data/model_exp_results.csv', 'w+') as outfile:
    all_files=glob.glob('data/embedded_model_data/*.csv.gz')
    for file in all_files:
        try:
            print()
            #run_data_through_models(file, outfile)
        except:
            print(f'{file} failed')

In [8]:
import glob